In [5]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

returns_path = '/content/drive/MyDrive/ProyectoCartera/data/processed/returns_daily.csv'
returns = pd.read_csv(returns_path, index_col=0, parse_dates=True)

# --- Verifica nombres de columnas e índice ---
print("Index:", returns.index.name)
print("Columns:", returns.columns)

# --- Limpiar cualquier fila o columna no numérica ---
returns = returns.apply(pd.to_numeric, errors='coerce')

# Reemplazar infinitos por NaN
returns = returns.replace([np.inf, -np.inf], np.nan)

# Eliminar filas con algún NaN
returns = returns.dropna(how='any')

print("Shape returns after cleaning:", returns.shape)
display(returns.head())

# Chequeos defensivos
assert not returns.isna().any().any(), "Hay NaNs en returns"
assert np.isfinite(returns.values).all(), "Hay infinitos en returns"

Mounted at /content/drive
Index: Price
Columns: Index(['EFA', 'LQD', 'VNQ', 'GOOGL', 'EEM', 'SPY', 'IWM', 'QQQ', 'IEF', 'MSFT',
       'AGG', 'AAPL', 'GLD', 'TLT', 'NVDA', 'GOOG', 'AMZN'],
      dtype='object')
Shape returns after cleaning: (2694, 17)


,EFA,LQD,VNQ,GOOGL,EEM,SPY,IWM,QQQ,IEF,MSFT,AGG,AAPL,GLD,TLT,NVDA,GOOG,AMZN
Price,,,,,,,,,,,,,,,,,
2015-01-05,-0.023605,0.004089,0.005473,-0.019054,-0.017797,-0.018059,-0.013369,-0.014669,0.006101,-0.009196,0.002173,-0.028172,0.015077,0.015709,-0.016890,-0.020846,-0.020517
2015-01-06,-0.011327,0.004072,0.009918,-0.024679,-0.004202,-0.009419,-0.017300,-0.013408,0.006718,-0.014677,0.002530,0.000094,0.011399,0.018017,-0.030319,-0.023177,-0.022833
2015-01-07,0.011115,0.001325,0.015331,-0.002941,0.021625,0.012461,0.012315,0.012891,-0.000185,0.012705,-0.000180,0.014022,-0.005891,-0.001975,-0.002605,-0.001713,0.010600
2015-01-08,0.013530,-0.003224,0.003775,0.003484,0.017037,0.017745,0.016962,0.019140,-0.004079,0.029418,-0.001533,0.038423,-0.004209,-0.013243,0.037618,0.003153,0.006836
2015-01-09,-0.004839,0.002653,0.000470,-0.012211,-0.003300,-0.008014,-0.009603,-0.006583,0.004934,-0.008405,0.002438,0.001072,0.011385,0.010952,0.004028,-0.012951,-0.011749


In [6]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.optimize import minimize

# --- Cargar retornos limpios ---
BASE = Path('/content/drive/MyDrive/ProyectoCartera')
returns_path = BASE / 'data' / 'processed' / 'returns_daily.csv'
returns = pd.read_csv(returns_path, index_col=0, parse_dates=True)

# Limpiar índice y datos
returns.index.name = None
returns = returns.apply(pd.to_numeric, errors='coerce')
returns = returns.replace([np.inf, -np.inf], np.nan).dropna(how='any')

print("Shape returns cleaned:", returns.shape)
display(returns.head())

# --- Calcular media de retornos y matriz de covarianza ---
mu = returns.mean() * 252           # anualizar media
cov = returns.cov() * 252           # anualizar covarianza
n = len(mu)

# --- Funciones de optimización ---
def sharpe_ratio(weights, mu, cov, risk_free_rate=0.0):
    port_return = np.dot(weights, mu)
    port_vol = np.sqrt(np.dot(weights.T, np.dot(cov, weights)))
    return - (port_return - risk_free_rate) / port_vol  # negativo porque minimize

# Restricciones: sum(weights) = 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

# Restricciones: long-only (0 <= weight <= 0.3 por activo, puedes cambiar 0.3)
bounds = tuple((0, 0.3) for _ in range(n))

# Inicialización
x0 = np.array([1/n]*n)

# --- Optimización ---
result = minimize(sharpe_ratio, x0, args=(mu, cov), method='SLSQP',
                  bounds=bounds, constraints=constraints)

if result.success:
    weights = result.x
    weights_dict = dict(zip(returns.columns, weights))
    print("Pesos óptimos (max Sharpe ratio):")
    for t, w in weights_dict.items():
        print(f"{t}: {w:.4f}")

    # Performance de la cartera
    port_return = np.dot(weights, mu)
    port_vol = np.sqrt(np.dot(weights.T, np.dot(cov, weights)))
    print(f"\nRetorno anual esperado: {port_return:.2%}")
    print(f"Volatilidad anual: {port_vol:.2%}")
    print(f"Sharpe ratio: {port_return/port_vol:.2f}")

else:
    print("Error en la optimización:", result.message)

Shape returns cleaned: (2694, 17)


,EFA,LQD,VNQ,GOOGL,EEM,SPY,IWM,QQQ,IEF,MSFT,AGG,AAPL,GLD,TLT,NVDA,GOOG,AMZN
2015-01-05,-0.023605,0.004089,0.005473,-0.019054,-0.017797,-0.018059,-0.013369,-0.014669,0.006101,-0.009196,0.002173,-0.028172,0.015077,0.015709,-0.016890,-0.020846,-0.020517
2015-01-06,-0.011327,0.004072,0.009918,-0.024679,-0.004202,-0.009419,-0.017300,-0.013408,0.006718,-0.014677,0.002530,0.000094,0.011399,0.018017,-0.030319,-0.023177,-0.022833
2015-01-07,0.011115,0.001325,0.015331,-0.002941,0.021625,0.012461,0.012315,0.012891,-0.000185,0.012705,-0.000180,0.014022,-0.005891,-0.001975,-0.002605,-0.001713,0.010600
2015-01-08,0.013530,-0.003224,0.003775,0.003484,0.017037,0.017745,0.016962,0.019140,-0.004079,0.029418,-0.001533,0.038423,-0.004209,-0.013243,0.037618,0.003153,0.006836
2015-01-09,-0.004839,0.002653,0.000470,-0.012211,-0.003300,-0.008014,-0.009603,-0.006583,0.004934,-0.008405,0.002438,0.001072,0.011385,0.010952,0.004028,-0.012951,-0.011749


Pesos óptimos (max Sharpe ratio):
EFA: 0.0000
LQD: 0.0000
VNQ: 0.0000
GOOGL: 0.0000
EEM: 0.0000
SPY: 0.0000
IWM: 0.0000
QQQ: 0.0000
IEF: 0.3000
MSFT: 0.0581
AGG: 0.0651
AAPL: 0.0265
GLD: 0.3000
TLT: 0.0000
NVDA: 0.1975
GOOG: 0.0052
AMZN: 0.0476

Retorno anual esperado: 21.07%
Volatilidad anual: 13.39%
Sharpe ratio: 1.57
